In [1]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (16, 100)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

In [2]:
%%javascript
// Disable auto-scrolling
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
%%R

# My commonly used R imports

require('tidyverse')

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.4     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Loading required package: tidyverse


In [4]:
%%R
# Load packages and prepare environment
library(tidyverse)
library(fixest)
library(ggplot2)


In [5]:
%%R
# Load the dataset
df <- read_csv("merged_hmda_with_demo.csv")

# Create binary outcome


Rows: 148917888 Columns: 28
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (5): lei, state_code, county_code, derived_dwelling_category, property_...
dbl (23): activity_year, action_taken, loan_purpose, lien_status, loan_amoun...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [77]:
%%R
nrow(df)

[1] 148917888


In [78]:
%%R
str(df)

spc_tbl_ [148,917,888 × 28] (S3: spec_tbl_df/tbl_df/tbl/data.frame)
 $ activity_year                    : num [1:148917888] 2015 2015 2015 2015 2015 ...
 $ lei                              : chr [1:148917888] NA NA NA NA ...
 $ state_code                       : chr [1:148917888] "6" "27" "41" "8" ...
 $ county_code                      : chr [1:148917888] "06065" "27147" "41009" "08014" ...
 $ derived_dwelling_category        : chr [1:148917888] NA NA NA NA ...
 $ action_taken                     : num [1:148917888] 1 6 1 1 6 3 1 1 6 4 ...
 $ loan_purpose                     : num [1:148917888] 1 3 1 3 1 1 3 1 1 3 ...
 $ lien_status                      : num [1:148917888] 1 4 1 1 4 1 1 1 4 1 ...
 $ loan_amount                      : num [1:148917888] 265 151 148 152 161 ...
 $ property_value                   : chr [1:148917888] NA NA NA NA ...
 $ occupancy_type                   : num [1:148917888] 1 2 1 2 1 1 1 1 1 1 ...
 $ income                           : num [1:148917888] 91 NA

In [8]:
%%R
df_clean <- df %>%
  filter(
    loan_purpose == 1,
    derived_dwelling_category == "Single Family (1-4 Units):Site-Built",
    occupancy_type == 1,
    income > 0
  ) %>%
  mutate(
    approved = ifelse(action_taken %in% c(1, 2, 6), 1, 0),
    debt_to_income = loan_amount / income,
    race = case_when(
      `applicant_race-1` == 1 ~ "White",
      `applicant_race-1` == 3 ~ "Black",
      TRUE ~ NA_character_
    )
  ) %>%
  filter(!is.na(race), is.finite(debt_to_income))


In [9]:
%%R
# Define the signature dates for anti-DEI laws per state
anti_dei_law_dates <- data.frame(
  state_code = c("ND", "OK", "TN", "IA", "ID", "FL", "MO", "MT", "NH", "OH"),
  law_date = as.Date(c(
    "2021-03-24",  # North Dakota
    "2021-05-05",  # Oklahoma
    "2021-05-25",  # Tennessee
    "2021-06-08",  # Iowa
    "2021-04-28",  # Idaho
    "2022-04-22",  # Florida
    "2023-05-12",  # Missouri
    "2023-04-19",  # Montana
    "2023-07-12",  # New Hampshire
    "2024-12-19"   # Ohio
  ))
)

In [10]:
%%R
# Merge the law dates into the main dataset
df_clean <- df_clean %>%
  left_join(anti_dei_law_dates, by = "state_code") %>%
  mutate(
    # Convert activity_year to Date format for comparison
    activity_date = as.Date(paste0(activity_year, "-01-01")),
    # Define post_dei_law based on the law_date
    post_dei_law = ifelse(!is.na(law_date) & activity_date >= law_date, 1, 0),
    # Define anti_dei_state as 1 if the state has a law_date
    anti_dei_state = ifelse(!is.na(law_date), 1, 0)
  )


In [11]:
%%R
df_clean <- df_clean %>%
  filter(`applicant_race-1` %in% c(1, 3)) %>%  # 1 = White, 3 = Black
  mutate(
    race = ifelse(`applicant_race-1` == 3, "Black", "White"),
    debt_to_income = loan_amount / income
  ) %>%
  filter(income > 0, is.finite(debt_to_income))


In [50]:
%%R
library(fixest)

# Make sure your data is filtered correctly first
df_black <- df_clean %>% filter(race == "Black")

# Run logistic regression with fixed effects
model_black_logit <- feglm(
  approved ~ post_dei_law * anti_dei_state * black + income + loan_amount + debt_to_income |
    activity_year + lei + county_code,
  data = df_black,
  family = "binomial"
)

# View summary
summary(model_black_logit)
#shivangi's feedback: add liberal or rebuplican and white versus non-white

GLM estimation, family = binomial, Dep. Var.: approved
Observations: 2,896,043
Fixed-effects: activity_year: 6,  lei: 3,671,  county_code: 2,645
Standard-errors: Clustered (activity_year) 
                                      Estimate   Std. Error   z value
post_dei_law                      -0.025734879 2.155201e-02 -1.194083
anti_dei_state                     0.615973820 2.928417e-01  2.103436
income                             0.000021822 1.322147e-05  1.650488
loan_amount                        0.000000377 8.432277e-08  4.472466
debt_to_income                    -0.000046777 1.472748e-05 -3.176202
post_dei_law:black                -0.000000110 1.345675e-07 -0.816825
anti_dei_state:black               0.000001969 3.971634e-07  4.957815
post_dei_law:anti_dei_state:black  0.000000110 0.000000e+00       Inf
                                    Pr(>|z|)    
post_dei_law                      2.3245e-01    
anti_dei_state                    3.5428e-02 *  
income                            

NOTES: 146,056 observations removed because of NA values (RHS: 146,056).
       0/1,077/317 fixed-effects (4,251 observations) removed because of only 0 (or only 1) outcomes.
The variables 'black' and 'post_dei_law:anti_dei_state' have been removed because of collinearity (see $collin.var).
In addition: Warning message:
Absence of convergence: Maximum number of iterations reached (25). Final deviance: 3,431,805.6. 


In [32]:
# %%R
# df_with_prediction <- df_clean %>% 
#     na.omit %>% 
#     mutate (
#         prediction=predict(model_full),
#         residuals=resid(model_full),
#         residuals_z=scale(residuals)
        
#     )%>%
#     arrange(residuals)
# write.csv(df_with_prediction, 'df_with_prediction.csv')


In [60]:
%%R
summary(model_black)

OLS estimation, Dep. Var.: approved
Observations: 2,900,294
Fixed-effects: activity_year: 6,  county_code: 2,962
Standard-errors: Clustered (activity_year) 
                          Estimate   Std. Error   t value   Pr(>|t|)    
post_dei_law         -0.0010498930 5.153188e-03 -0.203737 8.4659e-01    
anti_dei_state        0.1618609489 5.359695e-02  3.019966 2.9418e-02 *  
income                0.0000028332 2.247800e-07 12.604544 5.5824e-05 ***
loan_amount           0.0000000134 5.640000e-09  2.377879 6.3331e-02 .  
debt_to_income       -0.0000007375 3.101000e-07 -2.378364 6.3293e-02 .  
post_dei_law:black   -0.0000000462 1.827000e-08 -2.528512 5.2625e-02 .  
anti_dei_state:black  0.0000005168 7.602000e-08  6.798480 1.0483e-03 ** 
... 3 variables were removed because of collinearity (black, post_dei_law:anti_dei_state and post_dei_law:anti_dei_state:black)
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 0.460527     Adj. R2: 0.009115
                 Within R2:

In [68]:
%%R
df_clean %>%
  group_by(activity_year, race, anti_dei_state) %>%
  summarise(approval_rate = mean(approved)) %>%
  ggplot(aes(x = activity_year, y = approval_rate, color = race, linetype = as.factor(anti_dei_state))) +
  geom_line() +
  labs(title = "Approval Trends Over Time by Race and DEI Policy")
ggsave(
  filename = "approval_trends_all_states.png",
  width = 14,              
  height = 10,            
  dpi = 600
)



`summarise()` has grouped output by 'activity_year', 'race'. You can override
using the `.groups` argument.


In [63]:
%%R
df_clean %>%
  group_by(activity_year, race, anti_dei_state, state_code) %>%
  summarise(approval_rate = mean(approved, na.rm = TRUE)) %>%
  ggplot(aes(x = activity_year, y = approval_rate, color = race, linetype = as.factor(anti_dei_state))) +
  geom_line() +
  facet_wrap(~ state_code) +
  labs(
    title = "Approval Trends Over Time by Race and DEI Policy",
    x = "Year",
    y = "Approval Rate",
    linetype = "Anti-DEI State",
    color = "Race"
  ) +
  theme_minimal()
# Save 
ggsave(
  filename = "approval_trends_by_state.png",
  width = 14,              
  height = 10,            
  dpi = 600
)


`summarise()` has grouped output by 'activity_year', 'race', 'anti_dei_state'.
You can override using the `.groups` argument.


In [64]:
%%R

# Identify anti-DEI states
anti_dei_states <- df_clean %>%
  filter(anti_dei_state == 1) %>%
  distinct(state_code) %>%
  pull(state_code)

# Create a named vector for facet labels
state_labels <- df_clean %>%
  distinct(state_code) %>%
  mutate(label = if_else(state_code %in% anti_dei_states,
                         paste0(state_code, " *"),
                         state_code)) %>%
  deframe()


In [71]:
%%R
df_clean %>%
  group_by(activity_year, race, anti_dei_state, state_code) %>%
  summarise(approval_rate = mean(approved, na.rm = TRUE), .groups = "drop") %>%
  ggplot(aes(x = activity_year, y = approval_rate, color = race, linetype = as.factor(anti_dei_state))) +
  geom_line() +
  facet_wrap(~ state_code, labeller = labeller(state_code = state_labels), scales = 'free') +
  labs(
    title = "Approval Trends Over Time by Race and DEI Policy",
    x = "Year",
    y = "Approval Rate",
    linetype = "Anti-DEI State",
    color = "Race"
  ) +
  theme_minimal()
ggsave(
  filename = "approval_trends_by_state*.png",
  width = 14,              
  height = 10,            
  dpi = 600
)
